## Install packages

In [ ]:
!pip install "numpy<1.20"
!pip install "pyyaml<=5.3.1"
!pip install "scikit-learn<1.3"
!pip install mne
!pip install moabb
!pip install jupyter
!pip install seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 41.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
# !pip install mne
# !pip install MOABB
# !pip install pyriemann
# !pip install seaborn

# Verifiyng your installation

## Verify python environment

If your python environment is set up correctly, the following imports should succeed.

In [ ]:
import mne
import moabb
import pyriemann
import seaborn

## Verify MOABB Installation

Now let's check if MOABB is configured correctly.

In [ ]:
import os
import os.path as osp
from mne import get_config
from moabb.utils import set_download_dir

## Setting the download directory (optional)

Using MOABB, it is very easy to download and to extract EEG data from open datasets. Let's first select where we want to download the datasets. You can safely skip this part if you have no specific storage space requirements on your computer.

You can choose to change the download directory to any path of your choice. If the path/folder doesn’t exist, it will be created for you.

In [ ]:
original_path = get_config("MNE_DATA")
print(f"The download directory is currently {original_path}")
new_path = osp.join(osp.expanduser("~"), "mne_data") # change this to your desired path
# new_path = "/content/sample_data"
set_download_dir(new_path)

In [ ]:
check_path = get_config("MNE_DATA")
print(f"Now the download directory has been changed to {check_path}")

## Downloading or copying datasets

n case the internet connection is slow during the workshop, we have already prepared the datasets: several USB key are available and you could copy/paste the files directly on your computer.

Your should copy the datasets inside the mne_data that is in your home directory or in the folder you specified above. If you are not sure, please ask!

You should see at least the `MNE-ssvepexo-data` folder as listed below when you execute the following code:

In [ ]:
dataset_path = get_config("MNE_DATA")
print(f"The folder {check_path} contains:")
os.listdir(dataset_path)

Otherwise, you can download the dataset by executing the following:

In [ ]:
from moabb.datasets import SSVEPExo

datasets = [
    SSVEPExo(),
]
for dataset in datasets:
    dataset.download()
dataset_path = get_config("MNE_DATA")
print(f"The folder {check_path} contains:")
os.listdir(dataset_path)

# Discovering MNE and MOABB

## MNE
> \[MNE is an\] Open-source Python package for exploring, visualizing, and analyzing human neurophysiological data: MEG, EEG, sEEG, ECoG, NIRS, and more.

More information and documentation can be found at https://mne.tools/stable/index.html


### Downloading a dataset

To discover the features of MNE, we will make use of MOABB to download an SSVEP BCI dataset. MOABB internally uses MNE for data representation, so the downloaded data will be returned in an MNE format.

The dataset used here is the `SSVEPExo` dataset from the University of Versailles [1].

> The datasets contains recording from 12 male and female subjects aged between 20 and 28 years. Informed consent was obtained from all subjects, each one has signed a form attesting her or his consent. The subject sits in an electric wheelchair, his right upper limb is resting on the exoskeleton. The exoskeleton is functional but is not used during the recording of this experiment.
\
A panel of size 20x30 cm is attached on the left side of the chair, with 3 groups of 4 LEDs blinking at different frequencies. Even if the panel is on the left side, the user could see it without moving its head. The subjects were asked to sit comfortably in the wheelchair and to follow the auditory instructions, they could move and blink freely.
\
A sequence of trials is proposed to the user. A trial begin by an audio cue indicating which LED to focus on, or to focus on a fixation point set at an equal distance from all LEDs for the reject class. A trial lasts 5 seconds and there is a 3 second pause between each trial. The evaluation is conducted during a session consisting of 32 trials, with 8 trials for each frequency (13Hz, 17Hz and 21Hz) and 8 trials for the reject class, i.e. when the subject is not focusing on any specific blinking LED.
\
There is between 2 and 5 sessions for each user, recorded on different days, by the same operators, on the same hardware and in the same conditions.

https://neurotechx.github.io/moabb/generated/moabb.datasets.SSVEPExo.html#moabb.datasets.SSVEPExo

In [ ]:
from moabb.datasets import SSVEPExo

dataset = SSVEPExo()
dataset.download()
dataset.get_data()

Let's create a variable `raw` containing the raw, unprocessed data from the first subject in this experiment. MNE gives a nice overview of the metadata of this EEG recording.

In [ ]:
subj,session,run = 1, 'session_0', 'run_0'
raw = dataset.get_data(subjects=[subj])[subj][session][run]
raw

### Inspect raw data

MNE offers a lot of useful tools for data manipulation and visualization. You can plot the locations of the elctrodes used to record the data:

In [ ]:
sphere=(0,-25,0,100)
_ = raw.plot_sensors(show_names=True, kind='topomap', sphere=sphere)

Plot the raw EEG data:

In [ ]:
_ = raw.plot(duration=4, color={'eeg':'darkblue'}, scalings=dict(eeg=1e-2))

Or plot the power spectral density (psd). The psd applies a Fourier transform and visualizes the frequency content of the EEG signal. Notice the large peak at 50Hz (and its harmonic at 100=50\*2Hz) caused by noise from the power grid alternating at 50Hz, and the increase in activity from 8-12Hz indicating alpha activity in the EEG. Generally, the psd has a *1/f* characteristic, meaning that lower frequencies will have relatively higher power regardless of specific relevant brain activity occurring at these frequencies. Because no preprocessing was applied here, the SSVEP stimulation frequencies are not yet clearly visible from the psd.

In [ ]:
_ = raw.plot_psd(sphere=sphere)

Finally, to make preprocessing and classification easier, MNE can extract events that are recorded in a stimulation channel in the EEG. For SSVEP, these events signify the onsets of the stimulations of different targets. Events can also be used as classes for classifying the three SSVEP different frequencies.

In [ ]:
import numpy as np
from mne import pick_types

stim_idc = pick_types(raw.info, eeg=False, stim=True)
[raw.ch_names[i] for i in stim_idc]

In [ ]:
from mne import find_events
from mne.viz import plot_events

events = find_events(raw)
events

In [ ]:
_= plot_events(events)

## MOABB

The Mother Of All BCI Benchmarks is a package for interfacing with various EEG-BCI datasets from multiple paradigms and defines a framework to benchmark algorithms.

More information and documentation can be found at https://neurotechx.github.io/moabb/

### Paradigm

MOABB can implement standardized preprocessing pipelines that can be applied to multiple datasets to fairly compare these in a benchmark. MOABB calls a preprocessing pipeline a paradigm.

In [ ]:
from moabb.paradigms import SSVEP
dataset.event_id= {
    '13': 2,
    '17': 4,
    '21': 3,
    'rest': 1,
}
paradigm = SSVEP(fmin=10, fmax=25, n_classes=3)

### Pipelines

Classification algorithms can equally be defined independent of the dataset and are implemented using [Scikit-Learn pipelines](https://scikit-learn.org/stable/modules/compose.html#pipeline) to allow for multiple feature extraction, transformation and classification steps.

In [ ]:
from sklearn.pipeline import make_pipeline
from moabb.pipelines import SSVEP_CCA


interval = dataset.interval
freqs = paradigm.used_events(dataset)

pipelines = dict()
pipelines["CCA"] = make_pipeline(
    SSVEP_CCA(interval=interval, freqs=freqs, n_harmonics=3)
)

### Evaluation

Finally, when we have defined a paradigm and some pipelines, we can run the benchmark and evaluate BCI algorithm performance.

In [ ]:
from moabb.evaluations import WithinSessionEvaluation


evaluation = WithinSessionEvaluation(
    paradigm=paradigm,
    datasets=dataset,
    suffix='ssvep_workshop_moabb_intro',
    overwrite=True
)
results = evaluation.process(pipelines)
results

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure()
sns.catplot(kind='bar', x="score", y="subject", hue=None, data=results)

## References

[1] Emmanuel K. Kalunga, Sylvain Chevallier, Quentin Barthelemy. “Online SSVEP-based BCI using Riemannian Geometry”. Neurocomputing, 2016. arXiv report: https://arxiv.org/abs/1501.03227

# SSVEP Preprocessing (solution)

In the following notebook, you will implement some steps to clean up the recorded EEG data in order to extract interpretable frequency content for the different SSVEP classes. When in doubt, refer to the MNE documentation at https://mne.tools/stable/python_reference.html, or don't hesitate to ask a question or help each other out.

## Download the SSVEP Exoskeleton dataset

First, we again load the `SSVEPExo` dataset and extract the raw, unprocessed data into variable `raw`of datatype `mne.io.Raw`.

In [ ]:
from moabb.datasets import SSVEPExo

dataset = SSVEPExo()
dataset.download()
dataset.get_data()
subj,session,run = 3, 'session_0', 'run_0'
raw = dataset.get_data(subjects=[subj])[subj][session][run]
sphere=(0,-25,0,100)
raw

Without any preprocessing, we can already see distinct peaks at the frequencies of interest for this SSVEP paradigm (13Hz, 17Hz and 21Hz) in the power spectral density. However, this will not always be the case, and these peaks are overshadowed by the power of low frequencies and other peaks the alpha activity or the powerline noise.

## Rereference

Usually, the first step in EEG preprocessing is rereferencing. EEG data is recorded as differences in electrical potential between each electrode and a fixed reference electrode, which is not present in the recorded data. Afterwards, this reference can be changed in data processing to highlight some differences in the data and remove noise that is present in all electrodes.

**Output**: the `raw : mne.io.Raw` variable rereferenced using an apt method.

**Hint:** https://mne.tools/stable/generated/mne.io.Raw.html?highlight=set_eeg_reference#mne.io.Raw.set_eeg_reference

In [ ]:
import matplotlib.pyplot as plt

target_frequencies = [0,13,21,17]
fig, ax = plt.subplots(1,1,figsize=(17,6))
fig = raw.plot_psd(sphere=sphere,fmax=64,ax=ax, show=False)
for f in target_frequencies:
    ax.axvline(f, color='red', alpha=0.3)
fig.show()

In [ ]:
# Since no far-away electrodes that contain other brain activity
# but not the activity of interest are present, rereferencing can be ommitted here.
#_ = raw.set_eeg_reference('average')

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(17,6))
fig = raw.plot_psd(sphere=sphere,fmax=64,ax=ax, show=False)
for f in target_frequencies:
    ax.axvline(f, color='red', alpha=0.3)
fig.show()

## Filter

Next let's remove the powerline noise.

**Output**: the `raw : mne.io.Raw` variable filtered to drop the European powerline frequency and its harmonics.

**Hint**: https://mne.tools/stable/generated/mne.io.BaseRaw.html#mne.io.BaseRaw.notch_filternotch_filter#mne.io.BaseRaw.notch_filter

In [ ]:
powerline_freq = 50
# The data sample rate is 256Hz, hence the highest possible (Nyquist)
# frequency is 256Hz/2=128Hz.
# The maximum powerline harmonic we can use is thus 50Hz*2=100Hz < 128Hz
_ = raw.notch_filter([powerline_freq,powerline_freq*2])

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(17,6))
fig = raw.plot_psd(sphere=sphere,fmax=64,ax=ax, show=False)
for f in target_frequencies:
    ax.axvline(f, color='red', alpha=0.3)
fig.show()

## Cut epochs

The EEG is recorded as a continuous, multi-channel time series and is represented as such by the `mne.io.Raw` datatype. We are, however, only interested in the EEG activity during SSVEP stimulation, which might only occur sporadically troughout the recording. In order to drop irrelevant timepoints and end up with labeled segments of EEG data suited for analysis and classification, cut the continous signal into time segments (epochs) based on the events present in the stimulation channel.

In [ ]:
event_id = {
    'rest': 1,
    'stimulation/13Hz': 2,
    'stimulation/21Hz': 3,
    'stimulation/17Hz': 4,
}

Cut the continuous EEG signal in epochs each lasting for the entire duration of one SSVEP stimulation and including some pre-stimulation activity for baseline correction (see later).

**Output**: a variable `epochs : mne.Epochs` containing one epochs per event.

**Hint**: https://mne.tools/stable/generated/mne.Epochs.html

In [ ]:
from mne import find_events, Epochs

events = find_events(raw)
epochs = Epochs(raw, events=events, event_id=event_id, tmin=-3, tmax=5, baseline=(None,0), preload=True)

The psd should look a lot cleaner now and can be split up per stimulation class. Some harmonics should also be visible now, check if you can spot them.

In [ ]:
fig, axs = plt.subplots(1,4, figsize=(17,6), sharex=True, sharey=True)
for i,label in enumerate(event_id.keys()):
    axs[i].axvline(target_frequencies[i], color='red', alpha=0.3)
    epochs[label].plot_psd(
        sphere=sphere, tmin=0, fmax=64, show=False, ax=axs[i],
    )
    axs[i].set_title(label)
fig.show()

This looks nice, but the result is not really fit for analysis. For instance, the alpha activity and the lowest frequencies are still a lot more powerful than the frequencies of interest.

## Frequency band power feature extraction

### Time-frequency transform

To inspect which frequencies are prevalent in which classes and to allow for feature extraction, we can now plot frequency powers for the stimulation frequencies and their harmonics. We do this by applying a set of multitaper filters for the frequencies of interest to the data. If you want to gain a better understanding of EEG analysis in the time-frequency domain, please refer to Mike X Cohen's Analyzing Neural Time Series Data videos: https://www.youtube.com/channel/UCUR_LsXk7IYyueSnXcNextQ/playlists?view=50&sort=dd&shelf_id=1

In [ ]:
import numpy as np
from mne.time_frequency import tfr_multitaper, tfr_morlet

base_freqs = np.array([13]*9 + [17]*7 + [21]*6)
harmonics = np.array([1,2,3,4,5,6,7,8,9,1,2,3,4,5,6,7,1,2,3,4,5,6])
freqs = base_freqs*harmonics
n_cycles = freqs*3
epochs_tfr = tfr_morlet(epochs, freqs=freqs, n_cycles=n_cycles, average=False, return_itc=False)
epochs_tfr

In [ ]:
import seaborn as sns
import pandas as pd
from mne.epochs import make_metadata

mean_chan_power = epochs_tfr.data.mean(axis=(1,3))
sfreq = epochs.info['sfreq']
meta,_,_ = make_metadata(events, event_id, epochs.tmin, epochs.tmax, sfreq)
labels = meta['event_name']
df = pd.DataFrame(mean_chan_power.T, columns=labels)
df['Base frequency (Hz)'] = base_freqs
df['Harmonic'] = harmonics
df = df.melt(id_vars=['Base frequency (Hz)', 'Harmonic'], var_name='Label', value_name='Power (µV²)')
sns.catplot(kind='bar', col='Label', data=df, x='Base frequency (Hz)', hue='Harmonic', y='Power (µV²)', ax=axs[i], palette='magma')

### Baseline correction

Unfortunately, these features are not that easy to interpret and unfit to use for classification.
Ideally, the stimulation frequency of a class would be present as the frequency with the highest power.
There are several problems preventing this:
* The 1/f scaling characteristic makes it hard to visualize data across a wide frequency range. In the example above, the harmonics, while containing relevant information, fade away as they have increasingly higher frequencies.
* The 1/f scaling characteristic also prevents quantitative comparisons across frequencies. In the example above, the 1st harmonic of 13Hz has higher power than the first harmonic of 21Hz while 21Hz is being stimulated.
* Background noise at specific frequencies contaminates specific epochs and/or channels.
* Task-related changes in power in specific epochs and/or channels can be obscured by background activity or noise of higher power that is present regardless of the stimulation (e.g. alpha activity).
* It is hard to discern the trials where no targets were selected from the others.

To reveal the increase in frequency power generated by SSVEP stimulation, instead of using the raw power at specific frequencies as features, in each epoch, channel and frequency we want to look at increases or decreases of power relative to some baseline period in which no stimulation is happening. This will correct for the 1/f characteristic, and continuous noise present in some epochs or channels.

Choose a relevant baselining window and method and apply baselining to the timfe-frequency epochs. Does this work well for all subjects?

**Output**: the baseline corrected `epochs_tfr: mne.time_frequency.EpochsTFR` variable

**Hint**: https://mne.tools/stable/generated/mne.time_frequency.EpochsTFR.html#mne.time_frequency.EpochsTFR.apply_baseline

In [ ]:
epochs_tfr.apply_baseline((-2.5, -.5), mode='ratio')

Finally, after baseline correction, discard the baseline period so the resulting epochs contain only stimulated data.

**Output**: the cropped `epochs_tfr: mne.time_frequency.EpochsTFR` variable starting at 0s.

**Hint**: https://mne.tools/stable/generated/mne.time_frequency.EpochsTFR.html#mne.time_frequency.EpochsTFR.crop

In [ ]:
epochs.crop(tmin=0, tmax=5)

In [ ]:
mean_chan_power = epochs_tfr.data.mean(axis=(1,3))
mean_chan_power_db = 10*np.log10(mean_chan_power)

df = pd.DataFrame(mean_chan_power.T, columns=labels)
df['Base frequency (Hz)'] = base_freqs
df['Harmonic'] = harmonics
df = df.melt(id_vars=['Base frequency (Hz)', 'Harmonic'], var_name='Label', value_name='Power (dB)')
sns.catplot(kind='bar', col='Label', data=df, x='Base frequency (Hz)', hue='Harmonic', y='Power (dB)', ax=axs[i], palette='magma')

# SSVEP decoding algorithms

In this notebook, we'll take a look at some algorithms for decoding SSVEP responsens.
We'll make use of the `SSVEPExo` dataset and some existing state-of-the-art SSVEP decoding algorithms.
the dataset and the algorithms are accessed trough MOABB. You will also implement your own decoding algorithm, based on the feature extraction pipeline implemented in the previous notebook.

To be able to compare all algorithms, we'll only investigate the trials where stimulation is happening and do not attempt to classify the `rest` state.

In [ ]:
from moabb.datasets import SSVEPExo

dataset = SSVEPExo()
dataset.event_id= {
    '13': 2,
    '17': 4,
    '21': 3,
    'rest': 1,
}
interval = dataset.interval
sfreq=256

## Algorithm 1: Canonical Correlation Analysis

Canonical Correlation Analysis [2] is a fast and relatively performant algorithm that finds a spatial filter to isolate the EEG activity oscillating at the frequencies of interest. It works by constructing a spatial filter of which the output maximally correlates with sinusoidal template signals at the frequencies of interest and their harmonics. The code below plots an example of the template signals for the frequencies of interest and their first harmonic.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

base_freqs = [13,17,21]
harmonics = [1,2,3]
#harmonics = [1]

x = np.linspace(0, 2, sfreq*2)
i = 0
_,ax = plt.subplots(1,1,figsize=(17,6))
y_ticks = []
y_ticklabels = []
for f in base_freqs:
    for h in harmonics:
        ax.plot(x,np.sin(f*h*np.pi*x)+i*6)
        ax.plot(x,np.cos(f*h*np.pi*x)+i*6+2)
        y_ticks.append(i*6)
        y_ticks.append(i*6+2)
        y_ticklabels.append(f'sin({f}*{h}*π)')
        y_ticklabels.append(f'cos({f}*{h}*π)')
        i+=1
ax.set_yticks(y_ticks)
ax.set_yticklabels(y_ticklabels)
ax.set_title('CCA template signals for the 2 first harmonics')
ax.set_xlabel('Time (s)')


To use this algorithm, we first let MOABB construct a suiting preprocessing pipeline. The pipeline `paradigm_bandpass` will filter the data between 3Hz and 20Hz. To reduce computational load, we intend to use 2 harmonics to construct the template signals. The highest harmonic of the highest frequency 8.57Hz\*3=25,71Hz falls in this filter band.

In [ ]:
from moabb.paradigms import SSVEP

n_classes=3
paradigm_cca = SSVEP(fmin=3, fmax=30, n_classes=n_classes)

The CCA classifier is then constructed:

In [ ]:
from sklearn.pipeline import make_pipeline
from moabb.pipelines import SSVEP_CCA
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.cross_decomposition import CCA

pipelines_cca = dict()
pipelines_cca["CCA"] = make_pipeline(
    SSVEP_CCA(
        interval=interval,
        freqs=paradigm_cca.used_events(dataset),
        n_harmonics=3
    )
)

## Algorithm 2: Riemannian Geometry

Riemannian Geometry [3] is a relatively new and very performant method in biosignal classification.
In Riemannian Geometry classifiers, epochs are represented as covariance matrices. They contain a lot of information about the signal, and because they have some nice mathematical properties, they can be classified using optimalization on the Riemannian manifold. While the covariance matrices are highly dimensional, the assumption that they lie on a specific manifold greatly reduces the search space for an optimal solution.

The covariance matrix of a multichannel signal epoch of shape `(n_channels, n_times)` has shape `(n_channels, n_channels)`. Because SSVEP requires us to also include frequency information, we need some specific preprocessing which is taken care of by the `FilterBankSSVEP` processing paradigm.
To obtain SSVEP feature covariance matrices, `FilterBankSSVEP` applies a time-frequency transformation like in the previous notebook to the signal by applying multiple band-pass filters at the frequencies of interest and their harmonics. This results in signal epochs of shape `(n_channels, n_times, n_freqs*n_harmonics)`. MOABB provides an  `ExtendedSSVEPSignal` transformer, which flattens the epochs into shape `(n_chanels*n_freqs*n_harmonics, n_times)`. Finally, covariances of shape `(n_chanels*n_freqs*n_harmonics, n_chanels*n_freqs*n_harmonics)` can then be extracted and classified on the Riemannian manifold.

In [ ]:
from moabb.paradigms import FilterBankSSVEP

filter_freqs = np.outer(base_freqs,harmonics).flatten()
print(filter_freqs)
filters = [[f-.1, f+.1] for f in filter_freqs]

paradigm_rg = FilterBankSSVEP(n_classes=n_classes, filters=filters)

The code below plots the mean extracted covariance for each stimulation frequency. They show clear `(n_channels, n_channels)`-sized blocks around each target frequency and their harmonics. Note how MOABB's `FilterBankSSVEP`does not apply baseline correction in the time frequeny domain, causing the harmonics to have very weak covariance.

In [ ]:
from moabb.pipelines import ExtendedSSVEPSignal
from pyriemann.estimation import Covariances
from mne import concatenate_raws

subj,session = 3, 'session_0'
raw = concatenate_raws(list(dataset.get_data(subjects=[subj])[subj][session].values()))
X_tfr,labels,_ = paradigm_rg.process_raw(raw, dataset)
X_tfr_flat = ExtendedSSVEPSignal().fit_transform(X_tfr, labels)
scm = Covariances(estimator='scm')

fig, axs=plt.subplots(1,len(base_freqs), figsize=(17,6),sharex=True,sharey=True)
covs=np.zeros((len(base_freqs), X_tfr_flat.shape[1], X_tfr_flat.shape[1]))
for i,l in enumerate(base_freqs):
    covs[i] = np.mean(scm.fit_transform(X_tfr_flat[labels==str(l)]), axis=0)
vmax = np.max(np.abs(covs))
for i,l in enumerate(base_freqs):
    axs[i].imshow(covs[i],cmap=plt.get_cmap('RdBu_r'),vmin=-vmax, vmax=vmax)
    axs[i].set_title(f'{l}Hz')
    axs[i].grid(False)
    #ticks=np.arange(X_tfr.shape[3])*X_tfr.shape[1]+X_tfr.shape[1]/2
    #axs[i].set_xticks(ticks)
    #axs[i].set_yticks(ticks)
    #ticklabels=['13Hz', '13*2Hz', '13*3Hz','17Hz', '17*2Hz', '17*3Hz','21Hz', '21*2Hz', '21*3Hz' ]
    #axs[i].set_xticklabels(ticklabels)
    #axs[i].set_yticklabels(ticklabels)
fig.show()

Finally, we use `pyRiemann` to construct a classification pipeline that uses Riemannian Geometry to project these covariance matrices to a lower dimensional space (the tangent space). In this low dimensional space, a simple classifier like logistic regression can discriminate the classes.

In [ ]:
from pyriemann.tangentspace import TangentSpace
from sklearn.linear_model import LogisticRegression

pipelines_rg = dict()
pipelines_rg["RG+logreg"] = make_pipeline(
    ExtendedSSVEPSignal(),
    Covariances(estimator="lwf"),
    TangentSpace(),
    LogisticRegression(solver="lbfgs", multi_class="auto"),
)


## Algorithm 3: Frequency band power

In this part, you will reimplement the frequency band power feature extraction method with baselining you developed earlier in this workshop, this time in the form of a Scikit-Learn pipeline that can be used for classification. Choose a fitting preprocessing paradigm(bandpass `SSVEP`, or `FilterBankSSVEP`) and implement a scikit-learn pipeline that applies baseline correction to each frequency band of each channel of each epoch. As a final step in the pipeline, apply a suiting classifier to classify the extracted features.

**Hint**: You can implement [scikit-learn transformers](https://scikit-learn.org/stable/developers/develop.html)  with the following template:
```
class MyTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self, **params):
        ...
        
    def fit(self, X, y=None):
        ...
        return self
        
    def transform(self, X, y=None):
        ...
```

or by plugging your own function into a [FunctionTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html).

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
from sklearn.preprocessing import FunctionTransformer

paradigm_bandpower = FilterBankSSVEP(n_classes=n_classes, filters=filters, tmin=-3)

def power_and_baseline(X, y=None, sfreq=128, tmin=-3, baseline=(-2.5, -.5)):
    # Cut baselining window
    baseline_start = int((baseline[0]-tmin)*sfreq)
    baseline_end = int((baseline[1]-tmin)*sfreq)
    baseline_period = X[:,:,baseline_start:baseline_end,:]
    # Cut stimulation window
    stim_start = max(0, int(-tmin*sfreq))
    stimulation_period = X[:,:,stim_start:,:]
    # Calculate baseline power and stimulation power
    baseline_power = np.mean(baseline_period**2, axis=(1,2))
    stimulation_power = np.mean(stimulation_period**2, axis=(1,2))
    # Divide stimulation power by baseline power
    return np.log(stimulation_power/baseline_power)

pipelines_bandpower = dict()
pipelines_bandpower['power+logreg'] = make_pipeline(
    FunctionTransformer(power_and_baseline),
    LogisticRegression(solver="lbfgs", multi_class="auto"),
)

## Evaluate algorithm performance

You can run the following snippets to evaluate and compare the performance of your algorithms.

In [ ]:
from moabb.evaluations import WithinSessionEvaluation

evaluation_cca = WithinSessionEvaluation(
    paradigm=paradigm_cca,
    datasets=dataset,
    suffix="ssvep_workshop_cca",
    overwrite=False
)
results_cca = evaluation_cca.process(pipelines_cca)
results_cca

In [ ]:
evaluation_rg = WithinSessionEvaluation(
    paradigm=paradigm_rg,
    datasets=dataset,
    suffix="ssvep_workshop_rg",
    overwrite=False
)
results_rg = evaluation_rg.process(pipelines_rg)
results_rg

In [ ]:
evaluation_bandpower = WithinSessionEvaluation(
    paradigm=paradigm_bandpower,
    datasets=dataset,
    suffix="ssvep_workshop_bandpower",
    overwrite=True
)
results_bandpower = evaluation_bandpower.process(pipelines_bandpower)
results_bandpower

In [ ]:
import seaborn as sns
import pandas as pd

results = pd.concat([results_cca, results_rg, results_bandpower])
ax = sns.stripplot(data=results, y="score", x="pipeline", zorder=1, alpha=.5, palette="Set1")
ax = sns.pointplot(data=results, y="score", x="pipeline" , palette="Set1")
ax.set_ylabel("Accuracy")
ax.axhline(1/n_classes)
_ = ax.set_ylim(0,1)

## References

[2] Bin, G., Gao, X., Yan, Z., Hong, B., & Gao, S. (2009). An online multi-channel SSVEP-based brain-computer interface using a canonical correlation analysis method. Journal of neural engineering, 6(4), 046002. https://doi.org/10.1088/1741-2560/6/4/046002

[3] Kalunga, E. K., Chevallier, S., Barthélemy, Q., Djouani, K., Monacelli, E., & Hamam, Y. (2016). Online SSVEP-based BCI using Riemannian geometry. Neurocomputing, 191, 55-68. https://doi.org/10.1016/j.neucom.2016.01.007